## reconstituting 4D matrices from individual 3D matrices

In [1]:
import napari
#from aicsimageio import AICSImage
from skimage import io
from medpy.io import load #to read the ACME outputs we need to use medpy. aicsimageio doesn't seem to be able to handle it

In [2]:
from os import listdir,walk

In [3]:
import numpy as np
from math import ceil, floor

In [4]:
viewer = napari.Viewer(title='t26_acme_labelled_smad6')

C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\networkx\utils\backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [5]:
image_data, image_header = load('C:/Users/hkqur/Documents/napari/ACME_segmented/t26.czi_segmented.mha')

In [6]:
image_data.shape

(154, 168, 130)

In [7]:
viewer.add_image(image_data, name='t26')

<Image layer 't26' at 0x20625a8d580>

In [8]:
image_data.shape

(154, 168, 130)

In [9]:
pwd

'C:\\Users\\hkqur\\Documents\\napari'

In [10]:
directory_files = listdir('C:/Users/hkqur/Documents/napari/ACME_segmented/')
directory_files

['antpillmha',
 't18.czi_segmented.mha',
 't20.czi_segmented.mha',
 't22.czi_segmented.mha',
 't24.czi_segmented.mha',
 't26.czi_segmented.mha',
 't30.czi_segmented.mha',
 't32.czi_segmented.mha',
 't34.czi_segmented.mha',
 't36.czi_segmented.mha',
 't38.czi_segmented.mha',
 't40.czi_segmented.mha']

In [11]:
directory_images = [i for i in directory_files if i[len(i)-4:]=='.mha']

In [12]:
directory_images

['t18.czi_segmented.mha',
 't20.czi_segmented.mha',
 't22.czi_segmented.mha',
 't24.czi_segmented.mha',
 't26.czi_segmented.mha',
 't30.czi_segmented.mha',
 't32.czi_segmented.mha',
 't34.czi_segmented.mha',
 't36.czi_segmented.mha',
 't38.czi_segmented.mha',
 't40.czi_segmented.mha']

In [13]:
array_size_list = []
for a in range(0,(len(directory_images))):
    print('C:/Users/hkqur/Documents/napari/ACME_segmented/{}'.format(directory_images[a]))
    loaded_image, loaded_metadata = load('C:/Users/hkqur/Documents/napari/ACME_segmented/'+str(directory_images[a]))
    loaded_image_shape = loaded_image.shape
    array_size_list.append(loaded_image_shape)
    
print(array_size_list)
max_array_size = max(array_size_list)
    

C:/Users/hkqur/Documents/napari/ACME_segmented/t18.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t20.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t22.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t24.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t26.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t30.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t32.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t34.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t36.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t38.czi_segmented.mha
C:/Users/hkqur/Documents/napari/ACME_segmented/t40.czi_segmented.mha
[(154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 130), (154, 168, 123), (154, 168, 123), (154, 168, 123)]


In [14]:
# 2024-05-16: need to obtain biggest array and use that as template array that other smaller images need to pad up to. Just using the first image may not always work.
# reconstitute 4D image from disapparate ACME outputs so can all be labelling in one go in Napari.

array_3d_list = []
for a in range(0,(len(directory_images))):
    print('C:/Users/hkqur/Documents/napari/ACME_segmented/{}'.format(directory_images[a]))
    loaded_image, loaded_metadata = load('C:/Users/hkqur/Documents/napari/ACME_segmented/'+str(directory_images[a]))
    #print(loaded_image)
    array_x,array_y,array_z=loaded_image.shape
        
    if a==0: #begin 4D array by adding first image to it
        image_array = loaded_image
        #print(image_array)
        print(image_array.shape)
        first_image_array = image_array
        first_image_size_x,first_image_size_y,first_image_size_z = image_array.shape
        first_image_size = loaded_image.shape
        array_3d_list.append(loaded_image)
        
        
    else:
        if first_image_size == loaded_image.shape: #check if dimensions of previous image and current image are the same. if the same no processing needed, just add image to array and proceed
            print('loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
            #image_array = np.stack((image_array,loaded_image), axis=-1)
            array_3d_list.append(loaded_image)
        else: #if dimensions are mismatched, centre the images
            print('loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
            print('size mismatch')

            x_diff = abs(array_x-first_image_size_x)
            y_diff = abs(array_y-first_image_size_y)
            z_diff = abs(array_z-first_image_size_z)
            
            print('first_image_size_z = {} z_diff= {}'.format(first_image_size_z,z_diff))
            y_padding = ceil(y_diff/2)
            x_padding = ceil(x_diff/2)
            z_padding = ceil(z_diff/2)
            print(array_z,first_image_size_z)
            print('z_padding= ', z_padding)
            
            if z_diff%2!=0:
                z_pad_left = ceil(z_diff/2)
                z_pad_right = floor(z_diff/2)
                loaded_image = np.pad(loaded_image,((x_padding,x_padding),(y_padding,y_padding),(z_pad_left,z_pad_right)),'constant')
                
            else:       
                loaded_image = np.pad(loaded_image,((x_padding,x_padding),(y_padding,y_padding),(z_padding,z_padding)),'constant')
            #np.pad(loaded_image,((x_diff,x_diff),(y_diff,y_diff),(z_diff,z_diff)),'constant')
            print('following PAD- loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
            
            #image_array = np.stack((image_array,loaded_image), axis=-1)
            array_3d_list.append(loaded_image)
    

C:/Users/hkqur/Documents/napari/ACME_segmented/t18.czi_segmented.mha
(154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t20.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t22.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t24.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t26.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t30.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t32.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkqur/Documents/napari/ACME_segmented/t34.czi_segmented.mha
loaded_image: (154, 168, 130), image_array: (154, 168, 130)
C:/Users/hkq

In [15]:
image_array=np.array(array_3d_list)

In [16]:
image_array.shape

(11, 154, 168, 130)

In [17]:
viewer.add_image(image_array,name='image_array')

<Image layer 'image_array' at 0x2062769f910>

# to do

import the raw data to be overlaid by the imported ACME segmentation. Make sure that timepoints match up.